# German Christmas Movies
### Are there movies that people only watch on christmas? And if so, how can we identify them?

The idea is that we use Google Trends to identify "spikes" in searches for a particular movie during a set timeperiod. If we can find titles that stand out on christmas in the search trends, that would at least suggest people have increast intrest for that particular title on that day.

<b>Pytrends</b> is a libary that allows relativly easy access to the google trends api. https://github.com/GeneralMills/pytrends

install:

<b>pip install pytrends</b>

In [ ]:
%matplotlib inline
import pandas as pd
import datetime
from pytrends.request import TrendReq

In [ ]:
# hl specifies host language for accessing Google Trends
pytrends = TrendReq(hl="de", tz=-60)

In [ ]:
kw_list = ["FDP"]
pytrends.build_payload(kw_list, cat=0, timeframe="today 5-y", geo="DE", gprop="")

In [ ]:
df =pytrends.interest_over_time().drop(columns = ['isPartial'])
df.info()

In [ ]:
df.plot()

### Applying this to our question, we only need the information for a set timeframe

Update the kw_list for a potential christmas movie and set the timeframe for November - December. Pytrends requires a timeframe in the format "yyyy-mm-dd yyyy-mm-dd".

In [ ]:
kw_list = ["Stirb Langsam"]
startdate = "2018-11-01"
enddate = "2018-12-31"
tf = startdate + " " + enddate
tf

In [ ]:
pytrends.build_payload(kw_list, cat=0, timeframe=tf, geo="", gprop="")
df =pytrends.interest_over_time().drop(columns = ['isPartial'])
df.info()

In [ ]:
df.plot()

### Now we want the information for the last 5 years in that timeframe.

At this point it should be mentioned that the google api is not the benevolent type, if you send to many requests you will eventually get blocked. My best guess is that this point is reached after 1600 requests per day. After that I only got one request per minute.

So we should not request information we dont need.

In [ ]:
# Returns a DataFrame for a timeframe for the last X years starting 2019
def request_for_timeframe(years,s_month,s_day,e_month,e_day,title):
    startyear = 2019
    result = pd.Series()
    kw_list = [title]
    for i in range(years):
        # This is used to iterate over the years and only request the said timeframe.
        endyear = startyear - i
        startdate = str(endyear)+ "-" + s_month + "-" + s_day
        enddate = str(endyear)+ "-" + e_month + "-" + e_day
        tf = startdate + " " + enddate
        
        column_name = title +" " + str(endyear)
        pytrends.build_payload(kw_list, cat=0, timeframe=tf, geo="DE", gprop="")
        temp = pytrends.interest_over_time()
        result = result.append(temp.iloc[:,0])
    # The result was initially a Series, we transform it into a Dataframe
    result = result.to_frame(name = title)
    return result


In [ ]:
title = "Stirb Langsam"
year = 5
s_month = "10"
s_day = "01"
e_month = "12"
e_day = "31"

new_df = request_for_timeframe(year,s_month,s_day,e_month,e_day,title)
new_df

Now we can compare the trend data for all 5 years

In [ ]:
new_df.plot()

In [ ]:
import matplotlib.pyplot as plt


def plot_trend(new_df):
    fig, axes = plt.subplots(1,5, figsize=(15,3), dpi=120,  sharey = True)
    plt.style.use('bmh')
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off

    for i, ax in enumerate(axes.ravel()):
        ax.plot(new_df.loc[new_df.index.year == 2015+i])
        ax.set_title(str(2015+i))

        ax.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,       # ticks along the bottom edge are on
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
    try:
        plt.suptitle(new_df.columns[0], verticalalignment='bottom', fontsize=16)
    except:
        plt.suptitle(new_df.name, verticalalignment='bottom', fontsize=16)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_trend(new_df)

### Now comes the tricky part, we want to check these timeframes for for all the movies we can find. 

Here we need a little help. We need a set of movie titles that we can search for. Since all of our requests are actually 5 requests (one each year if we do 5 years) we can probably only do about 300 a day without getting restricted by the google trends api.

Imdb has a "Top Rated 250 Movies" list that fits quite nice here, as these are among the most popular movies anyways.

One option would be to use python to scrape these movies from Imdb, however these lists get used so often it is usually faster to just search for a csv file containing that information.

In [ ]:
#like so:

imdb_df = pd.read_csv("Data/imdb_full_movies.csv", header= 0, index_col = 0)
imdb_df.head(15)

Now for requesting all the information for these titles. This might take a while...

In [ ]:
# our result
result = pd.DataFrame()

# create a list of all the movie
movies = imdb_df["title"].tolist()
outputFile = "data/my_workshop_output.csv"

# This helps avoid requesting movies we have aready requested if we run the script a second time.

#result = pd.read_csv(outputFile, index_col = 0)
#checklist = result.columns.tolist()

In [ ]:
checklist[-1]

In [ ]:
len(checklist)

In [ ]:
("Der Pate" not in checklist)

In [ ]:
last_element = True
workshop_counter = 10
counter = 0

for movie in movies:
    if (counter == workshop_counter):
        break
    counter += 1
    #if (movie == checklist[-1]):
    #    last_element = True
    if (last_element):   
        # requesting the data for the current "movie"

        try:
            df = request_for_timeframe(year,s_month,s_day,e_month,e_day,movie)
            result = pd.concat([result,df], axis=1,join="outer")
            result.to_csv(outputFile, index = True)
        except:
            print("Error requesting: " + movie)
            pass
        # the most basic way to get feedback on how many titles we have so far
        print(len(result.columns))

In [ ]:
result = pd.read_csv("data/new_christmas_imdb - Backup.csv", index_col = 0)

In [ ]:
result.columns[:15]

In [ ]:
corrected_df = result

Getting the spikes.

In [ ]:
import datetime
corrected_df.index = pd.to_datetime(corrected_df.index)

In [ ]:
sum1 = corrected_df.loc[((corrected_df.index.month == 12) & ((corrected_df.index.day == 24) | (corrected_df.index.day == 25) | (corrected_df.index.day == 26)))].mean()
sum2 = corrected_df.mean()

#The highest difference between the average of the month and only the 13 indicate the biggest spike
r = sum1-sum2

r = r.sort_values(ascending = False)[:10]

In [ ]:
r

plotting the first

In [ ]:
r.index[0]

In [ ]:
for i in range(5):
    name = r.index[i]
    plot_trend(corrected_df[name])